In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder

In [2]:
path_d = '/Users/bumsoojoe/Desktop/Section4_project/csv/diabetes_prediction_dataset.csv'
path_m = '/Users/bumsoojoe/Desktop/Section4_project/csv/medical.csv'

In [3]:
dliabetes = pd.read_csv(path_d)
medical = pd.read_csv(path_m)

In [4]:
dliabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [5]:
medical.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# 당뇨 EDA
- taget : diabetes
-

In [6]:
dliabetes.shape

(100000, 9)

In [7]:
dliabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [8]:
dliabetes.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [9]:
dliabetes.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')

In [10]:
dliabetes.duplicated().sum()

3854

In [11]:
dliabetes[dliabetes.duplicated()].head(10)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
2756,Male,80.0,0,0,No Info,27.32,6.6,159,0
3272,Female,80.0,0,0,No Info,27.32,3.5,80,0
3418,Female,19.0,0,0,No Info,27.32,6.5,100,0
3939,Female,78.0,1,0,former,27.32,3.5,130,0
3960,Male,47.0,0,0,No Info,27.32,6.0,200,0
4106,Female,51.0,0,0,No Info,27.32,6.6,200,0
4128,Female,80.0,0,0,never,27.32,5.7,85,0
4224,Male,80.0,1,0,former,27.32,6.2,130,0
4263,Male,80.0,0,0,No Info,27.32,6.0,100,0
4486,Male,50.0,0,0,No Info,27.32,4.8,155,0


In [12]:
dli = dliabetes[dliabetes.duplicated()]
dli[(dli['age'] == 80.0)&(dli['blood_glucose_level']==159)&(dli['HbA1c_level']==6.6)]
# 값은 같지만 원전한 중복값이라고 보기 힘든 점은 존재한다. 하지마 머신러닝에 방해가 될 수도 있다. 이에 타겟 값의 비율이나 다른 값은 평균값에 영향이 큰지 확인 할 필요가 있다.

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
2756,Male,80.0,0,0,No Info,27.32,6.6,159,0
12440,Male,80.0,0,0,No Info,27.32,6.6,159,0
47366,Female,80.0,0,0,No Info,27.32,6.6,159,0
54794,Male,80.0,0,0,No Info,27.32,6.6,159,0
75961,Male,80.0,0,0,No Info,27.32,6.6,159,0


In [13]:
dliabetes['diabetes'].value_counts()

diabetes
0    91500
1     8500
Name: count, dtype: int64

In [14]:
dliabetes['smoking_history'].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

In [15]:
list_i = ['age','bmi','HbA1c_level','blood_glucose_level']
for i in list_i:
    print(f'Columns : {i}')
    print(dliabetes[i].mean())

Columns : age
41.885856
Columns : bmi
27.3207671
Columns : HbA1c_level
5.527507
Columns : blood_glucose_level
138.05806


In [16]:
test_d = dliabetes.copy()
test_d = test_d.drop_duplicates()
test_d = test_d.reset_index(drop=True)
test_d['diabetes'].value_counts()

diabetes
0    87664
1     8482
Name: count, dtype: int64

In [17]:
for i in list_i:
    print(f'Columns : {i}')
    print(test_d[i].mean())

Columns : age
41.79432571297817
Columns : bmi
27.32146111122668
Columns : HbA1c_level
5.532608740873255
Columns : blood_glucose_level
138.21823060761758


In [18]:
list_c = ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']
for i in list_c:
    print('Colums :',dliabetes[i].value_counts())
    print('===='*10)

Colums : gender
Female    58552
Male      41430
Other        18
Name: count, dtype: int64
Colums : hypertension
0    92515
1     7485
Name: count, dtype: int64
Colums : heart_disease
0    96058
1     3942
Name: count, dtype: int64
Colums : smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64
Colums : diabetes
0    91500
1     8500
Name: count, dtype: int64


In [19]:
for i in list_c:
    print('Colums :',test_d[i].value_counts())
    print('===='*10)

Colums : gender
Female    56161
Male      39967
Other        18
Name: count, dtype: int64
Colums : hypertension
0    88685
1     7461
Name: count, dtype: int64
Colums : heart_disease
0    92223
1     3923
Name: count, dtype: int64
Colums : smoking_history
never          34398
No Info        32887
former          9299
current         9197
not current     6367
ever            3998
Name: count, dtype: int64
Colums : diabetes
0    87664
1     8482
Name: count, dtype: int64


In [20]:
df_d = dliabetes.drop_duplicates()
df_d = df_d.reset_index(drop=True)
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36.0,0,0,No Info,24.60,4.8,145,0
96142,Female,2.0,0,0,No Info,17.37,6.5,100,0
96143,Male,66.0,0,0,former,27.83,5.7,155,0
96144,Female,24.0,0,0,never,35.42,4.0,100,0


In [21]:
df_d['smoking_history'].value_counts().index

Index(['never', 'No Info', 'former', 'current', 'not current', 'ever'], dtype='object', name='smoking_history')

# 의료보험 데이터셋

In [22]:
medical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [23]:
medical.shape

(1338, 7)

In [24]:
medical.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [25]:
medical.duplicated().sum()

1

In [26]:
df_m = medical.copy()
df_m = df_m.drop_duplicates()
df_m = df_m.reset_index(drop=True)
df_m

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [27]:
df_m.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [28]:
list_c = ['sex','children', 'smoker', 'region']
for i in list_c:
    print('Colums :',df_m[i].value_counts())
    print('===='*10)

Colums : sex
male      675
female    662
Name: count, dtype: int64
Colums : children
0    573
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64
Colums : smoker
no     1063
yes     274
Name: count, dtype: int64
Colums : region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64


In [29]:
list_c = ['age','bmi','charges']
for i in list_c:
    print('Colums :',i,df_m[i].mean())
    print('===='*10)

Colums : age 39.222139117427076
Colums : bmi 30.66345175766642
Colums : charges 13279.121486655948


# 데이터셋 EDA

In [30]:
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36.0,0,0,No Info,24.60,4.8,145,0
96142,Female,2.0,0,0,No Info,17.37,6.5,100,0
96143,Male,66.0,0,0,former,27.83,5.7,155,0
96144,Female,24.0,0,0,never,35.42,4.0,100,0


In [31]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96146 entries, 0 to 96145
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96146 non-null  object 
 1   age                  96146 non-null  float64
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking_history      96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.6+ MB


In [32]:
other = list(df_d[df_d['gender'] == 'Other'].index)
other

[12589,
 14730,
 16574,
 18540,
 23023,
 31547,
 33299,
 34393,
 34466,
 39626,
 54936,
 64780,
 65054,
 65347,
 66556,
 69075,
 73002,
 73408]

In [33]:
df_d.shape

(96146, 9)

In [34]:
df_d = df_d.drop(other)
df_d.shape

(96128, 9)

In [35]:
df_d[df_d['gender'] == 'Other']

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes


In [36]:
df_d = df_d.reset_index(drop=True)
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96123,Female,36.0,0,0,No Info,24.60,4.8,145,0
96124,Female,2.0,0,0,No Info,17.37,6.5,100,0
96125,Male,66.0,0,0,former,27.83,5.7,155,0
96126,Female,24.0,0,0,never,35.42,4.0,100,0


In [39]:
nan = list(df_d[df_d['smoking_history'] == 'No Info'].index)
df_d = df_d.drop(nan)
df_d = df_d.reset_index(drop=True)

In [40]:
df_d.shape

(63247, 9)

In [37]:
# 100% 비흡연자 vs 한 번이라도 흡연을 했다.
# 비흡연자(no) : 'never', 
# 흡연자(yes) : 'former', 'current', 'not current', 'ever'
'''
- 'never' 전혀 
- 'No Info' 정보없음 -> 결측치로 보고 제거
- 'former' 이전 
- 'current' 현재 
- 'not current' 현재는 아님 
- 'ever' 항상 
'''
smoking_100_yes = ['former', 'current', 'not current', 'ever']
smoking_100_No = ['never']


In [ ]:
def are_you_smoking(df,yes,no,new_df):
    new_df = df.copy()
    new_df['smoker'] = df.

In [ ]:
# 현재를 기준으로 비흡연자 vs 흡연자
# 비흡연자(no) : 'never', 'former', 'not current'
# 흡연자(yes) : 'ever','current'
smoking_yes = ['ever','current']
smoking_no = ['never', 'former', 'not current']

In [38]:
df_m

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500
